<a href="https://colab.research.google.com/github/Kraykray69/Bike_sharing-count-_ML/blob/main/Bike_sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install/Import Necessary Libraries


In [13]:
# Install ucimlrepo
!pip install ucimlrepo gradio scikit-learn

# Import the dataset
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import gradio as gr

Remove all warnings to make the output clean


In [14]:
import warnings
warnings.filterwarnings("ignore")

# Fetch The Dataset

In [15]:
# Cell 2: Fetch the Dataset
bike_sharing = fetch_ucirepo(id=275)
X = bike_sharing.data.features
y = bike_sharing.data.targets

# Data Preprocessing

In [16]:
# Cell 3: Data Preprocessing Function
def preprocess_data(X):
    # Drop unnecessary columns
    X_cleaned = X.drop(columns=['instant', 'dteday'], errors='ignore')

    # Separate categorical and numerical columns
    categorical_features = ['season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    numerical_features = ['temp', 'atemp', 'hum', 'windspeed']

    return X_cleaned, categorical_features, numerical_features

# Train & Evaluate Models

In [17]:
# Cell 4: Train and Evaluate Models Function
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'Random Forest': RandomForestRegressor(),
        'Linear Regression': LinearRegression(),
        'Support Vector Regressor': SVR(),
        'Decision Tree Regressor': DecisionTreeRegressor(),
        'K-Neighbors Regressor': KNeighborsRegressor()
    }

    results = []

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        results.append({
            'Model': name,
            'MSE': mse,
            'MAE': mae,
            'R2 Score': r2
        })

    return pd.DataFrame(results)


# Main Function

In [18]:
# Cell 5: Main Function to Execute the Pipeline
def main():
    # Preprocess data
    X_cleaned, categorical_features, numerical_features = preprocess_data(X)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y, test_size=0.2, random_state=42)


    # Train and evaluate models
    results = train_and_evaluate_models(X_train, X_test, y_train, y_test)

    # Save the best model (Random Forest) pipeline for prediction
    best_model = RandomForestRegressor()
    pipeline = Pipeline(steps=[('preprocessor', ColumnTransformer(
                                    transformers=[
                                        ('num', StandardScaler(), numerical_features),
                                        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
                                    ])),
                               ('model', best_model)])

    # Train the best model
    pipeline.fit(X_train, y_train)

    # Save the pipeline
    with open('best_model_pipeline.pkl', 'wb') as f:
        pickle.dump(pipeline, f)

    return results

# Cell 6: Run the Main Function and Display Results
results_table = main()

# Print the results outside the Gradio interface
print("Model Evaluation Results:")
print(results_table)

Model Evaluation Results:
                      Model           MSE         MAE  R2 Score
0             Random Forest   1766.525404   24.886271  0.944213
1         Linear Regression  19379.828368  104.803351  0.387981
2  Support Vector Regressor  19686.603575   89.643671  0.378293
3   Decision Tree Regressor   3344.868024   34.008774  0.894368
4     K-Neighbors Regressor   2777.113820   33.165880  0.912298


# Load Model

In [19]:
# Cell 7: Load Model Function
def load_model():
    try:
        model = pickle.load(open('best_model_pipeline.pkl', 'rb'))
        return model
    except Exception as e:
        return f"Error loading model: {str(e)}"

# Prediction

In [20]:
# Cell 8: Prediction Function
def predict(temp, atemp, hum, windspeed, season, mnth, hr, holiday, weekday, workingday, weathersit):
    model = load_model()

    # Prepare input data
    input_data = pd.DataFrame([[temp, atemp, hum, windspeed, season, mnth, hr, holiday, weekday, workingday, weathersit]],
                               columns=['temp', 'atemp', 'hum', 'windspeed', 'season', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit'])

    try:
        prediction = model.predict(input_data)
        return float(prediction[0])
    except Exception as e:
        return f"Error: {str(e)}"  # Return the error message


# Deployment

In [21]:
# Cell 9: Gradio Interface Function
def gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Bike Sharing Prediction")

        gr.Markdown("""
        ## Feature Explanations:

        - **Temperature (temp)**: Normalized temperature in Celsius (0 to 1).
        - **Feeling Temperature (atemp)**: Normalized temperature in Celsius adjusted for human perception (0 to 1).
        - **Humidity (hum)**: Normalized humidity (0 to 1).
        - **Windspeed (windspeed)**: Normalized windspeed (0 to 1).
        - **Season**:
            - 1: Spring
            - 2: Summer
            - 3: Fall
            - 4: Winter
        - **Month (mnth)**: Month of the year (1 to 12).
        - **Hour (hr)**: Hour of the day (0 to 23).
        - **Holiday**: Boolean value indicating if the day is a holiday (0 or 1).
        - **Weekday**: Day of the week (0=Sunday, 1=Monday,..., 6=Saturday).
        - **Working Day**: Boolean value indicating if the day is a working day (0 or 1).
        - **Weather Situation (weathersit)**:
            - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
            - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
            - 3: Light Snow, Rain + Thunderstorm + Scattered clouds, Rain + Scattered clouds
        """)

        temp = gr.Number(label="Normalized Temperature", value=0.5, precision=2)  # Example range 0-1
        atemp = gr.Number(label="Normalized Atemp", value=0.5, precision=2)  # Example range 0-1
        hum = gr.Number(label="Normalized Humidity", value=0.5, precision=2)  # Example range 0-1
        windspeed = gr.Number(label="Normalized Windspeed", value=0.5, precision=2)  # Example range 0-1
        season = gr.Dropdown(choices=[1, 2, 3, 4], label="Season (1=Spring, 2=Summer, 3=Fall, 4=Winter)")
        mnth = gr.Dropdown(choices=list(range(1, 13)), label="Month (1-12)")
        hr = gr.Number(label="Hour (0-23)", value=12)  # Example range 0-23
        holiday = gr.Checkbox(label="Holiday")
        weekday = gr.Dropdown(choices=list(range(0, 7)), label="Weekday (0=Sunday, 1=Monday,..., 6=Saturday)")
        workingday = gr.Checkbox(label="Working Day")
        weathersit = gr.Dropdown(choices=[1, 2, 3], label="Weather Situation (1=Clear, 2=Mist, 3=Light Snow)")

        submit_btn = gr.Button("Predict")
        output = gr.Textbox(label="Predicted Bike Count")

        def predict_and_display(*inputs):
            prediction = predict(*inputs)
            return f"Predicted Bike Count: {prediction}"

        submit_btn.click(predict_and_display,
                         inputs=[temp, atemp, hum, windspeed, season, mnth, hr, holiday, weekday, workingday, weathersit],
                         outputs=output)

    return demo

# Cell 10: Launch the Gradio App
demo = gradio_interface()
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1706d549a3d3d818fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
